# Restricted Boltzmann Machine for Small Dataset (ml-latest-small)

In [1]:
import csv
import numpy as np
from sklearn.neural_network import BernoulliRBM

# Obtaining the Dataset:
Before running this notebook, please download **ml-latest-small.zip** from https://grouplens.org/datasets/movielens/latest/ and extract the folder to the same directory.

## Loading Data

Movie ids in movies.csv are not sequential.

There are 9742 movies listed but the highest movie id is 193609.

In [2]:
movies = {}

with open('ml-latest-small/movies.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i != 0:
            m_id = int(row[0])
            movies[m_id] = row[1:]

# retrieves movie index based on movie id
m_index_lookup = {}
            
ordered = sorted(movies.items())
for i, (k, v) in enumerate(ordered):
    m_index_lookup[k] = i
    
n_movies = len(movies)
print("# of movies:", n_movies)

# of movies: 9742


In [3]:
# 3.0 is threshold for favorable rating
def convert_rating(rating: str):
    r = float(rating)
    if (r < 3.0):
        return 0
    return 1

users = []
curr_id = 0

with open('ml-latest-small/ratings.csv', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader):
        if i != 0:
            u_id = int(row[0])
            if u_id != curr_id:
                users.append(np.full(n_movies, -1))
                curr_id = u_id
            ratings = users[-1]
            m_id = int(row[1])
            m_index = m_index_lookup[m_id]
            m_rating = convert_rating(row[2])
            ratings[m_index] = m_rating

# convert to 2d numpy array
users = np.array(users)

print("users shape:", users.shape)

users shape: (610, 9742)


610 users with 9742 movie ratings each. Each rating is either 1(like), 0(dislike), or -1(missing)

In [4]:
def average_ratings_per_user(users):
    total = 0
    for user in users:
        # element-wise addition
        arr = user + 1
        # numpy has an advanced feature called Boolean Array Indexing
        # sets each element in arr that is greater than 0 to 1
        arr[arr > 0] = 1
        total += np.sum(arr)
    print("average # ratings per user:", total / len(users))
    
average_ratings_per_user(users)

average # ratings per user: 165.30491803278687


How will the RBM fare with such a sparse matrix of ratings?

### Processing Data (Optional)
**Try fitting the RBM without processing the data first**

In [5]:
def sort_rows(arr, col=0):
    return arr[np.argsort(arr[:, col])]

# selects k rows with the highest # of ratings
# TODO: update movie index
def filter(arr, k):
    arr2 = np.copy(arr)
    arr2 += 1
    arr2[arr2 > 0] = 1
    sums = np.empty((arr2.shape[0], 2))
    for i, row in enumerate(arr2):
        sums[i, 0] = np.sum(row)
        sums[i, 1] = i
    sums = sort_rows(sums)
    sums = sums[::-1] # reverse array
#     for i in range(k):
#         print(sums[i])
    indices = sums[:k, 1].T.astype(int)
    indices = np.sort(indices)
#     print(indices)
    top_k = arr[indices]
#     print(top_k.shape)
    return top_k

In [6]:
print("Before Filter:")
print(users.shape)
average_ratings_per_user(users)

print()
users2 = filter(users.T, 1000).T
print("After filtering top 1000 rated movies:")
print(users2.shape)
average_ratings_per_user(users2)

print()
users3 = filter(users2, 300)
print("After filtering top 300 user raters:")
print(users3.shape)
average_ratings_per_user(users3)

Before Filter:
(610, 9742)
average # ratings per user: 165.30491803278687

After filtering top 1000 rated movies:
(610, 1000)
average # ratings per user: 100.41967213114754

After filtering top 300 user raters:
(300, 1000)
average # ratings per user: 173.26


## Splitting data into training / testing sets

In [18]:
IMPUTE_MODE = "default"

In [19]:
# Some users are tough reviewers; others are more forgiving
# This function will randomly assign 1s and 0s based on their current review probabilities
def impute_missing(X, mode="default"):
    if mode == "default":
        for i, row in enumerate(X):
            # calculate how likely the user will give
            # a posive review
            neg = np.sum(row == 0)
            pos = np.sum(row == 1)
            p = pos / (pos + neg)

            missing = row == -1
            imputed = np.random.rand(np.sum(missing))
            imputed = imputed < p
            row[missing] = imputed.astype(int)
            
    elif mode == "random":
        for row in X:
            missing = row == -1
            imputed = np.random.rand(np.sum(missing))
            imputed = imputed < 0.5
            row[missing] = imputed.astype(int)
            
    elif mode == "zero":
        X[X < 0] = 0
        
def split_data(data, mode="default"):
    pi = np.random.permutation(data.shape[0])
    split = int(data.shape[0] * 0.8)

    Xtr_missing = data[pi[:split], :]
    Xte_missing = data[pi[split:], :]

    Xtr, Xte = np.copy(Xtr_missing), np.copy(Xte_missing)

    # Imputing Missing Values
    impute_missing(Xtr, mode)
    impute_missing(Xte, mode)
    
    return Xtr, Xte, Xtr_missing, Xte_missing


In [28]:
Xtr, Xte, Xtr_missing, Xte_missing = split_data(users3, mode=IMPUTE_MODE)

print("training shape:", Xtr.shape)
print("testing shape:", Xte.shape)

training shape: (240, 1000)
testing shape: (60, 1000)


## Fitting Data to RBM

In [62]:
rbm = BernoulliRBM(
    n_components = 16,
    learning_rate = 0.1,
    batch_size = 10,
    n_iter = 100,
    verbose = 0,
    random_state = 0
)

# this might take a minute or two
rbm.fit(Xtr)

BernoulliRBM(batch_size=10, learning_rate=0.1, n_components=16, n_iter=100,
             random_state=0, verbose=0)

### Likelihood?

This score fluctuates wildly based every time I run the rbm. 

In [56]:
print(rbm.score_samples(Xte).mean())

-326.5997556457192


In [57]:
import random

def sample_missing(rbm, Xobs, n_iters=10, mode="default"):
    Xhat = np.copy(Xobs)
    # impute missing values
    impute_missing(Xhat, mode)
    # print("preprocess done")
    for i in range(n_iters):
        Xhat = rbm.gibbs(Xhat).astype(int)
        Xhat[Xobs >= 0] = Xobs[Xobs >= 0]
    return Xhat

## Testing recommendations

Currently, Xte_missing has a handful (~165) of ratings per user, with the rest being missing. My plan is to conceal some of these known ratings and see whether or not the RBM can recover them.

In [58]:
# each user has rated at least 20 movies

# create mask marking all known ratings
mask = Xte_missing + 1
mask[mask > 0] = 1

Xte_concealed = np.copy(Xte_missing)
for user, mask_row in zip(Xte_concealed, mask):
    indices = mask_row.nonzero()[0]
    n_ratings = indices.shape[0]
    indices = np.random.permutation(indices)
    split = int(n_ratings * 0.3)
    # set 30% of known ratings to missing
    user[indices[:split]] = -1
    # turn off bits for the latter 70% part of the mask
    mask_row[indices[split:]] = 0
    
# Xte_concealed will now be same as Xte_missing 
#   but with 30% of known ratings concealed.
# mask will mark the locations of all ratings

In [59]:
Xte_predict = sample_missing(rbm, Xte_concealed, n_iters = 100, mode = IMPUTE_MODE)

In [60]:
num_checks = np.sum(mask)
arr1 = Xte_missing * mask
arr2 = Xte_predict * mask
result = arr1 ^ arr2
num_err = np.sum(result)

print("Errors:", num_err)
print("Total Checks:", num_checks)
print("Error Rate:", num_err / num_checks)
print("RMSE:", np.sqrt(num_err/num_checks))

Errors: 719
Total Checks: 2794
Error Rate: 0.2573371510379384
RMSE: 0.5072840930267165


Wow, this sucks

## More Evaluations

In [35]:
# When recommending movies to a user, it is better to not recommend good movies than to recommend bad movies
# False Positive: Recommend a movie when the user doesn't actually like the movie
# False Negative: Not recommend a movie when the user does like the movie

# In this case, I think False Positives are worse than False Negatives as it would cause the user to ignore
# the recommendation system

arr1_b = arr1.astype(bool)
arr2_b = arr2.astype(bool)

# false positive:
fp_arr = np.invert(arr1_b) & arr2_b
# false negative:
fn_arr = arr1_b & np.invert(arr2_b)
# true positive:
tp_arr = arr1_b & arr2_b

fp = np.sum(fp_arr)
fn = np.sum(fn_arr)
tp = np.sum(tp_arr)
# since we're checking only a handful of indices
# true negative has to be calculated from the 3 other values
tn = num_checks - (fp + fn + tp)

print("True Positives: ", tp)
print("True Negatives: ", tn)
print("False Positives:", fp)
print("False Negatives:", fn)

accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)

print()
print("Accuracy: ", accuracy)
print("Precision:", precision)
print("Recall:   ", recall)

True Positives:  2091
True Negatives:  63
False Positives: 417
False Negatives: 223

Accuracy:  0.7709377236936292
Precision: 0.833732057416268
Recall:    0.9036300777873811
